In [ ]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS `qwiklabs-gcp-02-3205ce60aeba.weather`;

DROP TABLE IF EXISTS `qwiklabs-gcp-02-3205ce60aeba.weather.weather_data`;
CREATE OR REPLACE EXTERNAL TABLE `qwiklabs-gcp-02-3205ce60aeba.weather.weather_data`
OPTIONS (
  format = 'CSV',
  uris = ['gs://labs.roitraining.com/data-to-ai-workshop/weather_data.csv'],
  skip_leading_rows = 1
);


Query is running:   0%|          |

""


In [ ]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [ ]:
%%bigquery
CREATE OR REPLACE MODEL `qwiklabs-gcp-02-3205ce60aeba.weather.gemini_model`
REMOTE WITH CONNECTION DEFAULT OPTIONS (
  ENDPOINT = 'gemini-2.0-flash'
);


Query is running:   0%|          |

""


In [ ]:
%%bigquery
SELECT
  ml_generate_text_llm_result AS generated_text
FROM
  ML.GENERATE_TEXT(
    MODEL `qwiklabs-gcp-02-3205ce60aeba.weather.gemini_model`,
    (
      SELECT
        CONCAT(
          'Generate a weather report or warning for ',
          city, ', ', state, ' on ', FORMAT_DATE('%B %d, %Y', date),
          '. Conditions: ', weather_condition,
          '. Temperature: ', CAST(temperature_f AS STRING), '°F. Wind Speed: ', CAST(wind_speed_mph AS STRING), ' mph. ',
          'Precipitation: ', CAST(precipitation_in AS STRING), ' in. Pressure: ', CAST(barometric_pressure_inHg AS STRING), ' inHg. ',
          'Humidity: ', CAST(humidity_percent AS STRING), '%.'
        ) AS prompt
      FROM
        `qwiklabs-gcp-02-3205ce60aeba.weather.weather_data`
    ),
    STRUCT(
      0.7 AS temperature,
      256 AS max_output_tokens,
      TRUE AS flatten_json_output
    )
  );

Query is running:   0%|          |

Downloading:   0%|          |

,generated_text
0,"## Atlanta, GA Weather Alert - February 18, 20..."
1,**URGENT WEATHER WARNING: HEAVY SNOW AND EXTRE...
2,"**URGENT WEATHER ALERT: MIAMI, FL - FEBRUARY 2..."
3,"**URGENT WEATHER WARNING - LOS ANGELES, CA - F..."
4,"**URGENT WEATHER ALERT: San Francisco, CA - Fe..."
...,...
295,"**San Francisco Weather Report - March 14, 202..."
296,"## Weather Alert for Atlanta, GA - March 16, 2..."
297,"**URGENT WEATHER ALERT FOR SEATTLE, WA - MARCH..."
298,"**URGENT WEATHER WARNING FOR ATLANTA, GA - MAR..."
